In [99]:
import heapq as hq

class Graph:

    def __init__(self, vertices):
        self.V = vertices
        self.graph = {}
        
        # store the minimum cycles of each node
        self.cycles = {}
        # store the dijkstra between any two node
        self.dijkstras = {}
                           
    # Dijkstra algorithm by heap data structure
    def dijkstra(self, start):
        distances = {}
        heap = [(0, start)]

        while heap:
            dist, node = hq.heappop(heap)
            if node in distances:
                continue  # Already encountered before
            # We know that this is the first time we encounter node.
            #   As we pull nodes in order of increasing distance, this 
            #   must be the node's shortest distance from the start node.
            distances[node] = dist
            for neighbor, weight in self.graph[node]:
                if neighbor not in distances:
                    hq.heappush(heap, (dist + weight, neighbor))

        return distances
    
    # Dijkstra algorithm for finding the minimum path between two node. 
    # During running algorithm, if the minimum path between two nodes defined, the algorithm will be terminated. 
    def dijkstraST(self, g, start, trg):
        distances = {}
        heap = [(0, start)]

        while heap:
            dist, node = hq.heappop(heap)
            
            # if we find the target node, terminate algorithm
            if node == trg:
                return dist
            
            if node in distances:
                continue  # Already encountered before
            # We know that this is the first time we encounter node.
            #   As we pull nodes in order of increasing distance, this 
            #   must be the node's shortest distance from the start node.
            distances[node] = dist
            for neighbor, weight in g[node]:
                if neighbor not in distances:
                    hq.heappush(heap, (dist + weight, neighbor))
                    
        # we could not find path between two nodes, so they are not connected. 
        return float('inf')

    # finding the path between any two node by Dijkstra algorithm
    # then store the result in self.dijkstras
    def AllDijkstra(self):
        for c in self.graph:
            self.dijkstras[c] = g.dijkstra(c)
        return self.dijkstras
    
    
    def leastCycleDijkstra(self, v):
        least_cycle = float('inf')
        for i in range(len(self.graph[v])):
            
            
            label = self.graph[v][i][0]
            weight = self.graph[v][i][1]
            index_delete1 = -1
            index_delete2 = -1
            
            temp_graph = {}
            for i in self.graph:
                temp_graph[i] = self.graph[i].copy()
                
            for i in range(len(temp_graph[v])):
                if temp_graph[v][i][0] == label:
                    index_delete1 = i
                    break
            del temp_graph[v][index_delete1]
            
            for i in range(len(temp_graph[label])):
                if temp_graph[label][i][0] == v:
                    index_delete2 = i
                    break
            del temp_graph[label][index_delete2]        
            
            temp = self.dijkstraST(temp_graph, v, label)
            
            cycle = temp + weight
            if cycle < least_cycle:
                least_cycle = cycle
            del temp_graph
            
        return least_cycle
    
    def find_lca(self, x, parents_x, y, parents_y):
        if parents_x == [] or x in parents_y:
            return x, []
        elif parents_y == [] or y in parents_x:
            return y, []
        elif len(parents_x) < len(parents_y):
            p_y = parents_y[:len(parents_x)]
            p_x = parents_x
        else:
            p_y = parents_y
            p_x = parents_x[:len(parents_y)]
        for i in range(len(p_y) -1, -1, -1):
            if p_y[i] == p_x[i]:
                return p_x[i], p_x[0: i-1]
        
        
    # the optimum method for finding girth of a graph
    def optimumCompositeDistance(self, start, graph):
        composite_distance = float('inf')
        distances = {}
        heap = [(0, start, [])]
        c = -1
        parents_c = [start]

        while heap:
            dist, node, parents = hq.heappop(heap)
            
            if node in distances or composite_distance <= dist:
                continue  # Already encountered before or we cannot find the minimum cycle after this node
                
            # We know that this is the first time we encounter node.
            #   As we pull nodes in order of increasing distance, this 
            #   must be the node's shortest distance from the start node.
            distances[node] = [dist, parents]
            
            for neighbor, weight in graph[node]:
                # a cycle detected
                
                if neighbor in distances and neighbor != parents[-1]:
                    
                    lca, parents_lca = self.find_lca(neighbor, distances[neighbor][1], node, parents)
                    
                    temp_cycle = dist+ weight+distances[neighbor][0] - distances[lca][0]
                    if temp_cycle < composite_distance:
                        composite_distance = temp_cycle
                        c = lca
                        parents_c = distances[lca][1]
                elif neighbor not in distances:
                    # print(neighbor, distances)
                    temp = parents + [node]
                    hq.heappush(heap, (dist + weight, neighbor, temp))
        
        
        print(composite_distance)
        return composite_distance, c, parents_c
    
    
    def optimumGirth(self):
        least_cycle = float('inf')
        temp_graph = {}
        for i in self.graph:
            temp_graph[i] = self.graph[i].copy()
        
        
        for node in self.graph.keys():
            if node in temp_graph:
                composite_distance, c, parents_c = self.optimumCompositeDistance(node, temp_graph)
                print(node, c)
                if node != c:
                    # theorem 2
                    # nodes between the node and the cycle should be deleted
                    for i in parents_c:
                        # edges adjacent to those nodes should be deleted as well. 
                        for j, weight in temp_graph[i]:
                            for k in range(0, len(temp_graph[j])):
                                if temp_graph[j][k][0] == i:
                                    del temp_graph[j][k]
                                    break

                        del temp_graph[i]
                print(temp_graph, '\n\n')
                if composite_distance < least_cycle:
                    least_cycle = composite_distance
        return least_cycle
        
        
    def AllCyclesDijkstra(self):
        
        for c in self.graph:
            self.cycles[c] = g.leastCycleDijkstra(c)
        return self.cycles
    
    def compositeDistance(self, x, c):
        return self.dijkstras[x][c] + self.cycles[c]
        
    def compositeDistanceX(self, x):
        least_composite = float('inf')
        for c in range(self.V):
            if self.compositeDistance(x, c) < least_composite:
                least_composite = self.compositeDistance(x, c)
        return least_composite
    
    def girthComposite(self):
        least_composite = float('inf')
        for x in range(self.V):
            cdx = self.compositeDistanceX(x)
            if  cdx < least_composite:
                least_composite = cdx
        return least_composite
    
    def girth(self):
        return min(self.cycles, key=self.cycles.get)
            

g = Graph(8)

g.graph = {
    "a": [("b", 8), ("c", 100), ("d", 6)],
    "b": [("a", 8), ("e", 5)],
    "c": [("a", 100), ("d", 2)],
    "d": [("a", 6), ("c", 2), ("e", 15), ("g", 3)],
    "e": [("b", 5), ("d", 15), ("f", 5)],
    "f": [("e", 5), ("g", 3), ("h", 6)],
    "g": [("d", 3), ("f", 3), ("h", 4)],
    "h": [("g", 4), ("f", 6)]
}



# print(g.AllDijkstra())

# print(g.dijkstra(0))
# d(x,y)= g.dijkstra(x)[y]

# print(g.AllCyclesDijkstra())

# l(c)= g.leastCycleForEachNode(c)
# g.leastCycleForEachNode(0)
# g.leastCycleDijkstra(2)

  
# d+(x,c) = d(x,c) + l(c)= g.compositeDistance(x, c)
# g.compositeDistance(0, 3)

# d+(x)= min d+(x, c) = g.compositeDistanceX(x)
# g.compositeDistanceX(0)

# print(g.girth())

# print(g.optimumCompositeDistance('b', g.graph)[0])
print(g.optimumGirth())


22
a g
{'b': [('e', 5)], 'c': [], 'e': [('b', 5), ('f', 5)], 'f': [('e', 5), ('g', 3), ('h', 6)], 'g': [('f', 3), ('h', 4)], 'h': [('g', 4), ('f', 6)]} 


23
b f
{'c': [], 'f': [('g', 3), ('h', 6)], 'g': [('f', 3), ('h', 4)], 'h': [('g', 4), ('f', 6)]} 


inf
c -1
{'f': [('g', 3), ('h', 6)], 'g': [('f', 3), ('h', 4)], 'h': [('g', 4), ('f', 6)]} 


13
f f
{'f': [('g', 3), ('h', 6)], 'g': [('f', 3), ('h', 4)], 'h': [('g', 4), ('f', 6)]} 


13
g g
{'f': [('g', 3), ('h', 6)], 'g': [('f', 3), ('h', 4)], 'h': [('g', 4), ('f', 6)]} 


13
h h
{'f': [('g', 3), ('h', 6)], 'g': [('f', 3), ('h', 4)], 'h': [('g', 4), ('f', 6)]} 


13
